# Project:  OpenStreetMap Data Wrangling with SQL

## OSM Data of Alpharetta, Georgia

### Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

>This map is the area I lived for one year after the undergradute degree. This expericen may help me to understand the result of database quering. I am interested to see what we can find out. The original dataset is from [OpenStreetMap](https://www.openstreetmap.org/relation/119572), which can be downloaded in XML format with the [OverpassAPI](https://overpass-api.de/api/map?bbox=-84.3601,33.9841,-84.2006,34.1634) option on the export menu. Here, we apply the data wrangling skill learned from the course to process the dataset. In general, the raw data will be cleaned and converted in CSV format. Then, the cleaned data will be transfered into SQLite. By using the tools in SQLite, we can get some statistics overview of the dataset.

In [1]:
#All libraries used in this project
import os
import os.path
import sys
import time
import requests
import pprint
import re
import codecs
import xml.etree.cElementTree as ET

import csv

import cerberus
import sqlite3
from collections import defaultdict

In [2]:
#Database schema module
sys.path.append('Schema//')
import schema_map_p1 as schema

<a id='wrangling'></a>
## Data Wrangling

>In this section, the raw data will be downloaded, cleaned and converted in CSV format. Then, the cleaned data will be imported into a SQLite database, which will be used in the next section.

### Steps
<ul>
<li><a href="#download">Download the Dataset</a></li>
<li><a href="#audit">Audit the Dataset</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='download'></a>
### Download the Dataset

>Download the original dataset from the OpenStreetMap by using the Overpass API option.

In [3]:
#The function used to download the original dataset
#It takes Overpass API url provided by OpenStreetMap and the filename used to store the file.  
def download_dataset(api_url, local_filename):
    total_size = 85557150 
    
    if os.path.exists(local_filename):
        if os.path.getsize(local_filename) == total_size:
            print 'Found and verified', local_filename
            return None

    r = requests.get(api_url, stream=True)

    with open(local_filename, 'wb') as f:
        print "Downloading %s" % local_filename, '...'
        
        dl_size = 0
        for chunk in r.iter_content(chunk_size=4096):
            if chunk:
                f.write(chunk)
                dl_size += len(chunk)
                done = int(100 * dl_size / float(total_size))
                sys.stdout.write("\r[%s%s]" % ('-' * done, ' ' * (100 - done)))    
                sys.stdout.flush()
        

    print '\nDownload finished. {} is ready.'.format(local_filename)

In [4]:
url = 'https://overpass-api.de/api/map?bbox=-84.3601,33.9920,-84.2004,34.1554'

osm_filename = 'data/alpharetta.osm'

In [6]:
download_dataset(url, osm_filename)

Found and verified data/alpharetta.osm


<a id='audit'></a>
### Audit the Dataset

>Download the original dataset from the OpenStreetMap by using the Overpass API option.

	```XML
	<tag k="tiger:name_base" v="Stonewall"/> 
	<tag k="tiger:name_direction_prefix" v="W"/> 
	<tag k="tiger:name_type" v="St"/>
	```

In [22]:
def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename, events=('start', )):
        if elem.tag not in tags:
            tags[elem.tag] = 1
        else:
            tags[elem.tag] += 1
    return tags

start_time = time.time()
tags = count_tags(osm_filename)
sorted_by_occurrence = [(k, v) for (v, k) in sorted([(value, key) for (key, value) in tags.items()], reverse=True)]

print 'Element types and occurrence of alpharetta.osm:\n'
pprint.pprint(sorted_by_occurrence)

print('\n--- %s seconds ---' % (time.time() - start_time))

Element types and occurrence of alpharetta.osm:

[('nd', 418468),
 ('node', 360143),
 ('tag', 204837),
 ('way', 40794),
 ('member', 4979),
 ('relation', 180),
 ('osm', 1),
 ('note', 1),
 ('meta', 1),
 ('bounds', 1)]

--- 2.53500008583 seconds ---


In [23]:
# From case study
def count_attrs(filename):
    attrs = {}
    for event, elem in ET.iterparse(filename, events=('start', 'end')):
        if event == 'end':
            for attr in elem.attrib:
                if attr not in attrs:
                    attrs[attr] = 1
                else:
                    attrs[attr] += 1
    return attrs

start_time = time.time()
attrs = count_attrs(osm_filename)
sorted_by_occurrence = [(k, v) for (v, k) in sorted([(value, key) for (key, value) in attrs.items()], reverse=True)]

print 'Attributes and occurrence of alpharetta.osm:\n'
pprint.pprint(sorted_by_occurrence)

print('\n--- %s seconds ---' % (time.time() - start_time))

Attributes and occurrence of alpharetta.osm:

[('ref', 423447),
 ('version', 401118),
 ('user', 401117),
 ('uid', 401117),
 ('timestamp', 401117),
 ('id', 401117),
 ('changeset', 401117),
 ('lon', 360143),
 ('lat', 360143),
 ('v', 204837),
 ('k', 204837),
 ('type', 4979),
 ('role', 4979),
 ('osm_base', 1),
 ('minlon', 1),
 ('minlat', 1),
 ('maxlon', 1),
 ('maxlat', 1),
 ('generator', 1)]

--- 3.28100013733 seconds ---


In [36]:
#From case study
def count_keys(filename):
    keys = {}
    for event, elem in ET.iterparse(filename, events=('start', 'end')):
        if event == 'end':
            key = elem.attrib.get('k')
            if key:
                if key not in keys:
                    keys[key] = 1
                else:
                    keys[key] += 1
    return keys

start_time = time.time()
keys = count_keys(osm_filename)
sorted_by_occurrence = [(k, v) for (v, k) in sorted([(value, key) for (key, value) in keys.items()], reverse=True)]

print 'Keys and occurrence in alpharetta.osm:\n'

#for i in range(10):
#    pprint.pprint(sorted_by_occurrence[i])

print('\nTotal number of attreibutes name found: %d' % len(sorted_by_occurrence))

pprint.pprint(sorted_by_occurrence)

print('\n--- %s seconds ---' % (time.time() - start_time))

Keys and occurrence in alpharetta.osm:


Total number of attreibutes name found: 312
[('building', 22983),
 ('addr:housenumber', 19004),
 ('addr:street', 18971),
 ('addr:city', 18938),
 ('highway', 17273),
 ('addr:postcode', 16568),
 ('source', 7459),
 ('name', 5508),
 ('tiger:county', 4428),
 ('tiger:cfcc', 4412),
 ('attribution', 4407),
 ('service', 4400),
 ('tiger:reviewed', 4230),
 ('tiger:name_base', 3763),
 ('tiger:name_type', 3499),
 ('Fulton:id', 3169),
 ('tiger:zip_left', 3010),
 ('tiger:zip_right', 2961),
 ('oneway', 2692),
 ('fcgis:STRUCT_USE', 1787),
 ('fcgis:YR_BUILT', 1784),
 ('fcgis:STORIES', 1784),
 ('fcgis:GFSF', 1784),
 ('fcgis:BLDG_FORM', 1784),
 ('fcgis:GEO_OID', 1782),
 ('surface', 1446),
 ('tiger:upload_uuid', 918),
 ('tiger:tlid', 918),
 ('tiger:source', 918),
 ('tiger:separated', 917),
 ('lanes', 844),
 ('LandPro08:LC', 785),
 ('LandPro08:LU', 784),
 ('LandPro08:LC_NAME', 784),
 ('LandPro08:LCLU', 784),
 ('LandPro08:DE5', 784),
 ('LandPro08:LU_NAME', 783),
 ('La

In [236]:
capital = re.compile(r'^([A-Z]|_)*$')
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
#double_colon = re.compile(r'^([a-zA-Z_0-9]|_)*:([a-zA-Z_0-9]|_)*:([a-zA-Z_0-9]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

In [237]:
def key_type(element, keys):
    if element.tag == "tag":
        
        if capital.search(element.attrib['k']):
            keys['capital'] += 1
        elif lower.search(element.attrib['k']):
            keys['lower'] += 1
        elif lower_colon.search(element.attrib['k']):
            keys['lower_colon'] += 1            
        elif problemchars.search(element.attrib['k']):
            keys['problemchars'] += 1
            print element.attrib['k']
        else:
            keys['other'] += 1
            #print element.attrib['k']
        
    return keys

In [238]:
def process_map(filename):
    keys = {'capital': 0, "lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

In [239]:
start_time = time.time()
keys = process_map(osm_filename)

print 'Key types and occurrence in alpharetta.osm:'
pprint.pprint(keys)

print('\n--- %s seconds ---' % (time.time() - start_time))

Key types and occurrence in alpharetta.osm:
{'capital': 349,
 'lower': 75328,
 'lower_colon': 105597,
 'other': 23563,
 'problemchars': 0}

--- 3.10800004005 seconds ---


In [89]:
def count_postcodes(filename):
    postcodes = {}
    for event, elem in ET.iterparse(filename, events=('start', 'end')):
        if event == 'end':
            key = elem.attrib.get('k')
            if key == 'addr:postcode':
                postcode = elem.attrib.get('v')
                if postcode not in postcodes:
                    postcodes[postcode] = 1
                else:
                    postcodes[postcode] += 1
    return postcodes


start_time = time.time()

postcodes = count_postcodes(osm_filename)
sorted_by_occurrence = [(k, v) for (v, k) in sorted([(value, key) for (key, value) in postcodes.items()], reverse=True)]

print 'Postcode values and occurrence in alpharetta.osm:\n'
pprint.pprint(sorted_by_occurrence)
print len(sorted_by_occurrence)
print('\n--- %s seconds ---' % (time.time() - start_time))

Postcode values and occurrence in alpharetta.osm:

[('30076', 9034),
 ('30022', 3773),
 ('30075', 1877),
 ('30004', 1425),
 ('30350', 417),
 ('30009', 24),
 ('30005', 13),
 ('58502', 1),
 ('30092', 1),
 ('30041', 1),
 ('30040', 1),
 ('300009', 1)]
12

--- 3.08200001717 seconds ---


In [88]:
def get_postcode(elem):
    if elem.tag in ['node', 'way', 'relation']:
        for tag in elem.iter():
            if tag.get('k') == 'addr:postcode':
                return True, tag.get('v')
        return False, None
    return False, None

In [109]:
def clean_postcode(filename, cleaned_filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    
    for child in ['node', 'way', 'relation']:
        for elem in root.findall(child):
            has_postcode, postcode_value = get_postcode(elem)
            if has_postcode:
                if postcode_value == '300009':
                    for tag in elem.iter():
                        if tag.get('k') == 'addr:postcode':
                            tag.set('v', '30009')

                elif postcode_value not in ['30004', '30005', '30009', '30022', '30040', 
                                          '30041', '30075', '30076', '30092', '30350']:
                    root.remove(elem)
 #tag.set('v', better_name) 4 5 9 22 76   
    return tree.write(cleaned_filename)

In [110]:
start_time = time.time()

cleaned_postcode = 'data/cleaned_postcode.xml'
clean_postcode(osm_filename, cleaned_postcode)


print('\n--- %s seconds ---' % (time.time() - start_time))


--- 13.6129999161 seconds ---


In [111]:
postcodes = count_postcodes(cleaned_postcode)
sorted_by_occurrence = [(k, v) for (v, k) in sorted([(value, key) for (key, value) in postcodes.items()], reverse=True)]

print 'Postcode values and occurrence after cleaning:\n'
pprint.pprint(sorted_by_occurrence)

Postcode values and occurrence after cleaning:

[('30076', 9034),
 ('30022', 3773),
 ('30075', 1877),
 ('30004', 1425),
 ('30350', 417),
 ('30009', 25),
 ('30005', 13),
 ('30092', 1),
 ('30041', 1),
 ('30040', 1)]


In [205]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Avenue", "Boulevard", "Commons", "Court", "Drive", 
            "Lane", "Parkway", "Place", "Road", "Square", 
            "Street", "Trail" ]

# UPDATE THIS VARIABLE
mapping = { "ave": "Avenue",
            "blvd":"Boulevard",
            'clse': 'Close',
            "ct": "Court",
            "dr": "Drive",
            "hwy": "Highway",
            "ln": "Lane",
            "lp": "Loop",
            "pkwy": "Parkway",
            "pl": "Place",
            'place': 'Place',
            "rd": "Road",
            "sq": "Square",
            "st": "Street",
            "ste": "Suite",
            'steeplechase': 'Steeplechase Drive',
            'trce': 'Trace',
            "trl": "Trail",
            "w": "West",
            "n": "North",
            "s": "South",
            "e": "East",            
            "ne": "Northeast",
            "se": "Southeast",
            "nw": "Northwest",
            "sw": "Southwest"}

In [206]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


In [207]:
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


In [208]:
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])

    return street_types

In [209]:
def update_name(name, mapping):
    name_part = name.split(',')[0]
    name_list = re.findall(r"[\w']+", name_part)
    end_of_street_name = len(name_list)
    
    if name_list[0].isdigit():
        del name_list[0]
    
    if name_list[0] == 'St':
        name_list[0] = 'Saint'
        return ' '.join(name_list)
    
    for i in range(len(name_list)):
        word = name_list[i].lower()
        if word in mapping:
            end_of_street_name = i
            name_list[i] = mapping[word]
        
    name_list = name_list[:(end_of_street_name+1)]
    better_name = ' '.join(name_list)
    return better_name

In [210]:
start_time = time.time()

street_types = audit(cleaned_postcode)
pprint.pprint(dict(street_types))
print len(street_types)
print('\n--- %s seconds ---' % (time.time() - start_time))

{'1': set(['Morris Road, 6th Floor, Building 1']),
 '1188': set(['North Point Circle, Space 1188']),
 'Alley': set(['Hidden Alley']),
 'Bay': set(['Benmore Bay']),
 'Bend': set(['Hambledon Bend']),
 'Bluff': set(['Chimney Bluff',
               'North Bluff',
               'North Eagles Bluff',
               'Tahoe Bluff']),
 'Chase': set(['Bluffwind Chase',
               'Brayward Chase',
               'Foxworth Chase',
               'Park Chase',
               'Shadow Creek Chase',
               'Taylor Meadow Chase']),
 'Chute': set(['Briers Chute']),
 'Circle': set(['Autry Landing Circle',
                'Autry Mills Circle',
                'Carriage Station Circle',
                'Cedar Knoll Circle',
                'Charleston Circle',
                'Chattahoochee Circle',
                'Crabapple Lake Circle',
                'Crestview Circle',
                'Dunwoody Creek Circle',
                'East Creek Circle',
                'Eves Circle',
          

In [211]:
start_time = time.time()

for st_type, ways in street_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        if name != better_name:
            print name, "=>", better_name

print('\n--- %s seconds ---' % (time.time() - start_time))

Merganser Ln => Merganser Lane
Providence Place Way => Providence Place
Morris Road, 6th Floor, Building 1 => Morris Road
Birmingham Hwy => Birmingham Highway
Chenery Dr => Chenery Drive
Steeplechase => Steeplechase Drive
6000 North Point Pkwy => North Point Parkway
Norcross St => Norcross Street
North Point Circle, Space 1188 => North Point Circle
Harris Commons place => Harris Commons Place
N Hillbrooke Trce => North Hillbrooke Trace
Post Oak Clse => Post Oak Close

--- 0.00399994850159 seconds ---


In [212]:
def clean_street_name(filename, cleaned_filename):
    tree = ET.parse(filename)
    root = tree.getroot()

    for tag in root.findall('*/tag'):
        if is_street_name(tag):
            name = tag.get('v')
            better_name = update_name(name, mapping)
            tag.set('v', better_name)

    return tree.write(cleaned_filename)


--- 13.3680000305 seconds ---


In [213]:
start_time = time.time()

cleaned_street_name = 'data/cleaned_street_name.xml'
clean_street_name(cleaned_postcode, cleaned_street_name)

print('\n--- %s seconds ---' % (time.time() - start_time))


--- 13.3459999561 seconds ---


In [219]:
start_time = time.time()
street_types = audit(cleaned_street_name)

for st_type, ways in street_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        if name != better_name:
            print name, "=>", better_name
print('\n--- %s seconds ---' % (time.time() - start_time))


--- 3.26600003242 seconds ---


In [215]:

count_number = 0
for _, element in ET.iterparse(cleaned_street_name, ('start', 'end')):
    if  element.tag == "tag":
        if element.attrib['k'] == 'phone':
            count_number += 1

print 'Total count of phone numbers:', count_number

Total count of phone numbers: 116


In [216]:
digit_only = re.compile(r'\d+')

In [221]:
def number_type(element, number_types):
    if element.tag == "tag":
        if element.attrib['k'] == 'phone':
            if digit_only.match(element.attrib['v']):
                number_types['digit_only'] += 1
            else:
                number_types['other'] += 1
                print element.attrib['v']
        
    return number_types

In [222]:
def process_map(filename):
    number_types = {"digit_only": 0, "other": 0}
    for _, element in ET.iterparse(filename, ('start', 'end')):
        number_types = number_type(element, number_types)

    return number_types

In [223]:
start_time = time.time()
pprint.pprint(process_map(cleaned_street_name))
print('\n--- %s seconds ---' % (time.time() - start_time))

+1 770 993 9031
+1 770 993 9031
+1-404-754-9290
+1-404-754-9290
+1-770-346-9119
+1-770-346-9119
(770) 284-3101
(770) 284-3101
+1 (678) 319-0018
+1 (678) 319-0018
+1 (770) 410-1122
+1 (770) 410-1122
(770) 642-0395
(770) 642-0395
(770) 992-1617
(770) 992-1617
(770) 552-1390
(770) 552-1390
+1 404 452 9263
+1 404 452 9263
(888) 660-5890
(888) 660-5890
+1 770 3469880
+1 770 3469880
+1 770 5968985
+1 770 5968985
+1 (770) 569-7298
+1 (770) 569-7298
(770) 817-4656
(770) 817-4656
+1-470-282-5400
+1-470-282-5400
+1-770-360-8014
+1-770-360-8014
+1-678-777-6385
+1-678-777-6385
+1-404-587-4573
+1-404-587-4573
+1 (678) 393-8333
+1 (678) 393-8333
+1-770-587-4595
+1-770-587-4595
+1-770-475-4101
+1-770-475-4101
(770) 691-0637
(770) 691-0637
(404) 369-1302
(404) 369-1302
+1 770 740 7050
+1 770 740 7050
+1-770-772-9033
+1-770-772-9033
+1 (678) 280-0550
+1 (678) 280-0550
(678) 393-0101
(678) 393-0101
+1 770 3607766
+1 770 3607766
+1 (678) 461-7900
+1 (678) 461-7900
+1-770-993-0533
+1-770-993-0533
(770) 99

In [225]:
def clean_phone_number(filename, cleaned_filename):
    tree = ET.parse(filename)
    root = tree.getroot()

    for tag in root.findall('*/tag'):
        if tag.attrib['k'] == 'phone':
            phone_number = tag.get('v')
            if not digit_only.match(phone_number):
                update_phone_number = re.sub(r'[\D|\s]+', '', phone_number)[-10: ]
                tag.set('v', update_phone_number)

    return tree.write(cleaned_filename)

In [226]:
start_time = time.time()

cleaned_phone_number = 'data/cleaned_phone_number.xml'
clean_phone_number(cleaned_street_name, cleaned_phone_number)

pprint.pprint(process_map(cleaned_phone_number))

print('\n--- %s seconds ---' % (time.time() - start_time))

{'digit_only': 116, 'other': 0}

--- 16.8329999447 seconds ---


In [254]:
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema
DATADIR = 'Prepared CSV\\'

In [255]:
NODES_PATH = os.path.join(DATADIR, "nodes.csv")
NODE_TAGS_PATH = os.path.join(DATADIR, "nodes_tags.csv")
WAYS_PATH = os.path.join(DATADIR, "ways.csv")
WAY_NODES_PATH = os.path.join(DATADIR, "ways_nodes.csv")
WAY_TAGS_PATH = os.path.join(DATADIR, "ways_tags.csv")

In [243]:
# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

In [244]:
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  

    if element.tag == 'node':
        for attrib in element.attrib:
            if attrib in NODE_FIELDS:
                node_attribs[attrib] = element.attrib[attrib]
        
        for child in element:
            node_tag = {}
            if LOWER_COLON.match(child.attrib['k']):
                node_tag['type'] = child.attrib['k'].split(':',1)[0]
                node_tag['key'] = child.attrib['k'].split(':',1)[1]
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
            elif PROBLEMCHARS.match(child.attrib['k']):
                continue
            else:
                node_tag['type'] = 'regular'
                node_tag['key'] = child.attrib['k']
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child.attrib['v']
                tags.append(node_tag)
        
        return {'node': node_attribs, 'node_tags': tags}
        
    elif element.tag == 'way':
        for attrib in element.attrib:
            if attrib in WAY_FIELDS:
                way_attribs[attrib] = element.attrib[attrib]
        
        position = 0
        for child in element:
            way_tag = {}
            way_node = {}
            
            if child.tag == 'tag':
                if LOWER_COLON.match(child.attrib['k']):
                    way_tag['type'] = child.attrib['k'].split(':',1)[0]
                    way_tag['key'] = child.attrib['k'].split(':',1)[1]
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                elif PROBLEMCHARS.match(child.attrib['k']):
                    continue
                else:
                    way_tag['type'] = 'regular'
                    way_tag['key'] = child.attrib['k']
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child.attrib['v']
                    tags.append(way_tag)
                    
            elif child.tag == 'nd':
                way_node['id'] = element.attrib['id']
                way_node['node_id'] = child.attrib['ref']
                way_node['position'] = position
                position += 1
                way_nodes.append(way_node)
        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


In [245]:
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

In [246]:
def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


In [263]:
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [261]:
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""
 #codecs.
    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

In [262]:
start_time = time.time()
if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(cleaned_phone_number, validate=True)
print('\n--- %s seconds ---' % (time.time() - start_time))


--- 803.332999945 seconds ---


In [276]:
#NODES_PATH = os.path.join(DATADIR, "nodes.csv")
#NODE_TAGS_PATH = os.path.join(DATADIR, "nodes_tags.csv")
#WAYS_PATH = os.path.join(DATADIR, "ways.csv")
#WAY_NODES_PATH = os.path.join(DATADIR, "ways_nodes.csv")
#WAY_TAGS_PATH = os.path.join(DATADIR, "ways_tags.csv")
#os.path.getsize(local_filename)
start_time = time.time()
print('File size of nodes.csv: %.2f MB' % (float(os.path.getsize(NODES_PATH))/1000000))
print('File size of nodes_tags.csv: %.2f MB' % (float(os.path.getsize(NODE_TAGS_PATH))/1000000))
print('File size of ways.csv: %.2f MB' % (float(os.path.getsize(WAYS_PATH))/1000000))
print('File size of ways_nodes.csv: %.2f MB' % (float(os.path.getsize(WAY_NODES_PATH))/1000000))
print('File size of ways_tags.csv: %.2f MB' % (float(os.path.getsize(WAY_TAGS_PATH))/1000000))
print('\n--- %s seconds ---' % (time.time() - start_time))

File size of nodes.csv: 34.23 MB
File size of nodes_tags.csv: 1.07 MB
File size of ways.csv: 2.93 MB
File size of ways_nodes.csv: 10.27 MB
File size of ways_tags.csv: 6.40 MB

--- 0.000999927520752 seconds ---


In [280]:
conn = sqlite3.connect("OSM_alpharetta.db")
conn.text_factory = str
cur = conn.cursor()

In [288]:
# create nodes table
start_time = time.time()

cur.execute("Drop TABLE nodes;")
cur.execute("CREATE TABLE nodes (id, lat, lon, user, uid, version, changeset, timestamp);")
with open(NODES_PATH, 'rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) \
             for i in dr]

cur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) \
                VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)
conn.commit()

print('\n--- %s seconds ---' % (time.time() - start_time))


--- 1.85599994659 seconds ---


In [289]:
#create nodes_tags table
start_time = time.time()

cur.execute("Drop TABLE nodes_tags;")
cur.execute("CREATE TABLE nodes_tags (id, key, value, type);")
with open(NODE_TAGS_PATH, 'rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO nodes_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
conn.commit()

print('\n--- %s seconds ---' % (time.time() - start_time))


--- 0.15299987793 seconds ---


In [291]:
#Create ways table
start_time = time.time()

cur.execute("Drop TABLE ways;")
cur.execute("CREATE TABLE ways (id, user, uid, version, changeset, timestamp);")
with open(WAYS_PATH, 'rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("INSERT INTO ways (id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db)
conn.commit()

print('\n--- %s seconds ---' % (time.time() - start_time))


--- 0.245000123978 seconds ---


In [294]:
#Create ways_nodes table
start_time = time.time()

cur.execute("Drop TABLE ways_nodes;")
cur.execute("CREATE TABLE ways_nodes (id, node_id, position);")
with open(WAY_NODES_PATH, 'rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]

cur.executemany("INSERT INTO ways_nodes (id, node_id, position) VALUES (?, ?, ?);", to_db)
conn.commit()

print('\n--- %s seconds ---' % (time.time() - start_time))


--- 1.35100007057 seconds ---


In [295]:
#Create ways_tags table
start_time = time.time()

cur.execute("CREATE TABLE ways_tags (id, key, value, type);")
with open(WAY_TAGS_PATH, 'rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO ways_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
conn.commit()

print('\n--- %s seconds ---' % (time.time() - start_time))


--- 0.650000095367 seconds ---


In [343]:
start_time = time.time()

print('File size of OSM_alpharetta.db: %.2f MB' % (float(os.path.getsize('OSM_alpharetta.db'))/1000000))

print('\n--- %s seconds ---' % (time.time() - start_time))

File size of OSM_alpharetta.db: 62.68 MB

--- 0.0 seconds ---


In [299]:
def number_of_nodes():
    result = cur.execute('SELECT COUNT(*) FROM nodes')
    return result.fetchone()[0]

In [302]:
start_time = time.time()
print("Number of nodes: %d" % number_of_nodes())
print('\n--- %s seconds ---' % (time.time() - start_time))

Number of nodes: 360142

--- 0.0569999217987 seconds ---


In [303]:
def number_of_ways():
    result = cur.execute('SELECT COUNT(*) FROM ways')
    return result.fetchone()[0]

In [305]:
start_time = time.time()
print("Number of ways: %d" % number_of_ways())
print('\n--- %s seconds ---' % (time.time() - start_time))

Number of ways: 40794

--- 0.00500011444092 seconds ---


In [320]:
def number_of_unique_users():
    result = cur.execute(
        'SELECT COUNT(DISTINCT(all_rec.uid)) \
         FROM (SELECT uid FROM nodes \
               UNION ALL \
               SELECT uid FROM ways) all_rec')
    return result.fetchone()[0]

In [321]:
start_time = time.time()
print("Number of unique users: %d" % number_of_unique_users())
print('\n--- %s seconds ---' % (time.time() - start_time))

Number of unique users: 317

--- 0.148999929428 seconds ---


In [338]:
def top_contributing_users():
    users = []
    for row in cur.execute(
        'SELECT all_rec.user, COUNT(*) as num \
         FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) all_rec \
         GROUP BY all_rec.user \
         ORDER BY num DESC \
         LIMIT 10'):
        users.append(row)
    return users

In [339]:
start_time = time.time()

top_users = top_contributing_users()
print "Top contributing users: "
for i in range(len(top_users)):
    print top_users[i]
print('\n--- %s seconds ---' % (time.time() - start_time))

Top contributing users: 
('Saikrishna_FultonCountyImport', 201863)
('Liber', 38321)
('woodpeck_fixbot', 37322)
('demory', 12372)
('Jack the Ripper', 10202)
('afonit', 8751)
('TeresaPeteti', 8038)
('greenv505', 7608)
('mackerski', 5916)
('ranjithjoy', 4974)

--- 0.208999872208 seconds ---


In [340]:
def number_of_users_contributing_once():
    result = cur.execute(
        'SELECT COUNT(*) \
         FROM (SELECT all_rec.user, COUNT(*) as num \
               FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) all_rec \
               GROUP BY all_rec.user \
               HAVING num=1) u')
    return result.fetchone()[0]

In [341]:
start_time = time.time()
print("Number of users contributing once: %d" % number_of_users_contributing_once())
print('\n--- %s seconds ---' % (time.time() - start_time))

Number of users contributing once: 55

--- 0.214999914169 seconds ---


In [389]:
def common_ammenities():
    result = []
    for row in cur.execute('SELECT value, COUNT(*) num \
            FROM nodes_tags \
            WHERE key="amenity" \
            GROUP BY value \
            ORDER BY num DESC \
            LIMIT 10'):
        result.append(row)
    return result

In [390]:
start_time = time.time()

top_ammenities = common_ammenities()
print "Top contributing users: "
for i in range(len(top_ammenities)):
    print top_ammenities[i]
print('\n--- %s seconds ---' % (time.time() - start_time))

Top contributing users: 
('school', 47)
('restaurant', 41)
('place_of_worship', 31)
('fast_food', 15)
('grave_yard', 14)
('fire_station', 11)
('bench', 10)
('fuel', 10)
('cafe', 8)
('bank', 7)

--- 0.00299978256226 seconds ---


In [370]:
def religion():
    result = []
    for row in cur.execute('SELECT value, count(*) num FROM nodes_tags \
                WHERE key = "religion" \
                GROUP BY value \
                ORDER BY num'):
        result.append(row)
    return result

In [371]:
start_time = time.time()

religion = religion()
print "Religion in the area: "
for i in range(len(biggest_religion)):
    print biggest_religion[i]
print('\n--- %s seconds ---' % (time.time() - start_time))

Religion in the area: 
('christian', 31)

--- 0.0019998550415 seconds ---


In [420]:
def popular_cuisines():
    result = []
    for row in cur.execute('SELECT value, COUNT(*) as num \
                FROM nodes_tags \
                WHERE key="cuisine" \
                GROUP BY value \
                ORDER BY num DESC \
                LIMIT 3'):
        result.append(row)
    return result

In [421]:
start_time = time.time()

popular_cuisines = popular_cuisines()
print "Popular cuisines in the area: "
for i in range(len(popular_cuisines)):
    print popular_cuisines[i]
print('\n--- %s seconds ---' % (time.time() - start_time))
#problem occurs when american & American

Popular cuisines in the area: 
('mexican', 6)
('burger', 5)
('chicken', 3)

--- 0.00300002098083 seconds ---


In [417]:
result = []
for row in cur.execute('SELECT * FROM nodes_tags \
            WHERE value = "coffee"'):
    result.append(row)
result

[]

In [422]:
#Some problem here
result = []
for row in cur.execute('Select n.value, count(*) \
            from nodes_tags n, (SELECT id FROM nodes_tags \
                                WHERE value = "coffee_shop") i\
            WHERE n.id = i.id \
              AND n.key = "name" \
            GROUP BY n.value'):
    result.append(row)
result

[('Starbucks', 2)]

In [433]:
#Find the address of all Wendy's in the area
result = []
for row in cur.execute('Select w2.value from nodes_tags n1, nodes_tags n2, ways_tags w1, ways_tags w2 \
                        Where n1.value = "McDonald\'s" \
                          And n1.id = n2.id \
                          And n2.key = "housenumber" \
                          And n2.value = w1.value \
                          And w1.id = w2.id \
                          And w2.key = "street"'):
    result.append(row)
result

[('Windward Plaza',),
 ('Newgate Court',),
 ('Bellingrath Boulevard',),
 ('Darien Park Drive',),
 ('Crabapple Lake Court',),
 ('Roxburgh Drive',),
 ('Dover Avenue',),
 ('Jenkins Court',),
 ('Dodds Grove Lane',)]

In [423]:
result = []
for row in cur.execute('SELECT id FROM nodes_tags WHERE value = "Starbucks"'):
    result.append(row)
result

[('2714938157',),
 ('2810148148',),
 ('4285520360',),
 ('4524912377',),
 ('5369193298',)]

In [430]:
result = []
for row in cur.execute('Select * FROM nodes_tags \
            WHERE id = "2714938157" '):
    result.append(row)
result

[('2714938157', 'city', 'Roswell', 'addr'),
 ('2714938157', 'housenumber', '10800', 'addr'),
 ('2714938157', 'postcode', '30076', 'addr'),
 ('2714938157', 'street', 'Alpharetta Highway', 'addr'),
 ('2714938157', 'amenity', 'cafe', 'regular'),
 ('2714938157', 'cuisine', 'coffee_shop', 'regular'),
 ('2714938157', 'internet_access', 'wlan', 'regular'),
 ('2714938157', 'name', 'Starbucks', 'regular'),
 ('2714938157', 'phone', '7705521390', 'regular'),
 ('2714938157', 'website', 'http://www.starbucks.com/', 'regular')]

In [416]:
result = []
for row in cur.execute('Select * FROM nodes_tags \
            WHERE id = "4141785063"'):
    result.append(row)
result

[('4141785063', 'amenity', 'fast_food', 'regular'),
 ('4141785063', 'name', 'Burger King', 'regular')]

In [436]:
result = []
for row in cur.execute('Select w2.value \
                        from nodes n, nodes_tags n1, nodes_tags n2, \
                             ways w, ways_tags w1, ways_tags w2 \
                        Where n1.value = "McDonald\'s" \
                          And n1.id = n2.id \
                          And n1.id = n.id \
                          And n.uid = w.uid \
                          And n2.key = "housenumber" \
                          And n2.value = w1.value \
                          And w1.id = w2.id \
                          And w.id = w1.id \
                          And w2.key = "street"'):
    result.append(row)
result

[]

In [442]:
result = []
for row in cur.execute('Select strftime("%Y",timestamp) year FROM nodes limit 3'):
    result.append(row)
result

[('2013',), ('2009',), ('2009',)]

In [452]:
result = []
for row in cur.execute('Select i.year, count(*) num \
                        From (Select strftime("%Y",timestamp) year FROM nodes \
                              Union ALL\
                              Select strftime("%Y",timestamp) year FROM ways) i\
                        Group by year \
                        Order by year desc'):
    result.append(row)
result

[('2018', 29934),
 ('2017', 25556),
 ('2016', 56028),
 ('2015', 159539),
 ('2014', 17698),
 ('2013', 12013),
 ('2012', 4965),
 ('2011', 637),
 ('2010', 1710),
 ('2009', 91390),
 ('2008', 591),
 ('2007', 875)]

In [451]:
result = []
for row in cur.execute('Select value, count(*) num FROM ways_tags \
                        where key ="street" group by value order by num desc limit 5'):
    result.append(row)
result

[('Roxburgh Drive', 155),
 ('Arborwoods Drive', 108),
 ('Martin Road', 91),
 ('Nesbit Lakes Drive', 89),
 ('Six Branches Drive', 79)]